## 1. Consultas sobre redis (local)

In [1]:
# Conexion a redis 
import redis 

REDIS_PORT = 6379
r = redis.Redis(host='localhost',
                port=REDIS_PORT,
                decode_responses=True)

In [6]:
# Para comprobar cuanta memoria usada, habria que utilizarla tanto por maestro como replicas  
from typing import Dict, Any

mem: Dict[str, Any] = r.info("memory") #type: ignore
used_mb = mem["used_memory"] / 1024 / 1024
total_mb = mem.get("total_system_memory", 0) / 1024 / 1024

print(f"Usada: {used_mb:.2f} MB")
print(f"Total sistema: {total_mb:.2f} MB")

Usada: 1.40 MB
Total sistema: 7567.28 MB


### 1.1 Colas de pacientes

Se guardan listas con prioridad de pacientes para atenderlos.

In [7]:
# Scan de las colas de pacientes
zsets = r.scan(match="ED-*:DEP-*", #type: ignore
       _type="ZSET")[1]
for key in zsets:
       print(key)

ED-5:DEP-ONCOLOGIA
ED-4:DEP-TRAUMATOLOGIA
ED-1:DEP-ONCOLOGIA
ED-3:DEP-ONCOLOGIA
ED-1:DEP-NEUROLOGIA


In [ ]:
# Comprobar el contenido de las colas añadidas:
for key in zsets:
    print(f"Contenido de: {key}")
    scan_results = r.zscan(key,match="*")
    print(f"  Database --> {scan_results[0]}",end="\n    ") #type: ignore
    for patient in scan_results[1]: #type: ignore
        print(patient, end=" ")
    print("\n")

Contenido de: ED-3:DEP-ONCOLOGIA
  Database --> 0
    ('Daniel', 1.0) ('Mariana', 27.0) ('Mateo', 32.0) ('Alejandro', 41.0) ('Camila', 82.0) ('Diego', 92.0) ('Lucas', 93.0) ('Sofía', 99.0) 

Contenido de: ED-4:DEP-NEUROLOGIA
  Database --> 0
    ('Isabella', 35.0) ('Javier', 38.0) ('Daniel', 43.0) ('Mateo', 45.0) ('Lucas', 58.0) ('Valentina', 83.0) ('Mariana', 89.0) ('Alejandro', 100.0) 

Contenido de: ED-3:DEP-CARDIOLOGIA
  Database --> 0
    ('Lucía', 0.0) ('Diego', 27.0) ('Mateo', 30.0) ('Isabella', 38.0) ('Javier', 50.0) ('Martina', 59.0) ('Andrés', 61.0) ('Sofía', 86.0) 

Contenido de: ED-2:DEP-PEDIATRIA
  Database --> 0
    ('Valentina', 4.0) ('Daniel', 12.0) ('Camila', 25.0) ('Martina', 57.0) ('Mariana', 63.0) ('Alejandro', 67.0) ('Mateo', 67.0) ('Diego', 72.0) 

Contenido de: ED-2:DEP-ONCOLOGIA
  Database --> 0
    ('Lucas', 14.0) ('Javier', 27.0) ('Mariana', 42.0) ('Andrés', 54.0) ('Isabella', 64.0) ('Sofía', 72.0) ('Lucía', 91.0) ('Martina', 95.0) 



In [ ]:
# Pacientes prioritarios en cada cola 
for key in zsets: 
    print(key)
    [(patient, score)] = r.zrevrange(key, start=0, end=0, withscores=True) #type: ignore
    print(f"-> Paciente: {patient} | Prioridad: {score}")

ED-3:DEP-ONCOLOGIA
-> Paciente: Sofía | Prioridad: 99.0
ED-4:DEP-NEUROLOGIA
-> Paciente: Alejandro | Prioridad: 100.0
ED-3:DEP-CARDIOLOGIA
-> Paciente: Sofía | Prioridad: 86.0
ED-2:DEP-PEDIATRIA
-> Paciente: Diego | Prioridad: 72.0
ED-2:DEP-ONCOLOGIA
-> Paciente: Martina | Prioridad: 95.0


### 1.2 Alertas de salas

En las salas habra una dinamica pub/sub para que cuando suceda algo de relevancia en la sala (generalmente relacionado con problemas) se notifique instantaneamente.

Por la tematica pub/sub, la ingesta es dinamica, por lo que se incluira directamente aqui en el notebook. La idea es que se lanzara un hilo que publique alertas mientras el hilo principal se queda a la escucha. 

**NOTA**: Para detener la ejecicion, pulsar el boton de detener celda (en mi vscode se encuentra en la esquiza superior izquierda mientras tengas la celda seleccionada).

In [26]:
import threading
import time
import random
from typing import List

# Numeros de salas 
salas = [random.randint(0,20) for i in range(8)]

# Numeros de edificios 
edificios = [random.randint(0,5) for i in range(8)]

# Canales
# Formato de clave: ALERTAS:EDIFICIO:SALA
canales = list(zip(edificios, salas))
canales = [f"ALERTAS:ED-{edificio}:SALA-{sala}" for (edificio, sala) in canales]

# Funcion de ejemplo de publisher    
def publisher(canales: List[str]):

    factores = ["Temperatura", "Frecuencia cardiaca", "Glucosa en sangre",
                        "Frecuencia respiratoria", "Diuresis", "Nivel de conciencia"]

    posibles_alertas = ["Alta", "Baja"]

    # Vamos a meter 2 alertas por canal
    for canal in canales:
        for i in range(2):
            alerta = f"{factores[random.randint(0, len(factores))-1]}: {posibles_alertas[random.randint(0, len(posibles_alertas)-1)]}"
            r.publish(canal, alerta)
            time.sleep(0.5)
    print("[PUB] Todas las alertas enviadas!")

# Hilo principal: Subscriber
try:
    # Creamos el objeto tipo pub sub
    sub = r.pubsub()
    # Nos suscribimos a todos los canales
    for canal in canales:
        sub.subscribe(canal)
        
    # Creamos el hilo del publisher
    pub_thread = threading.Thread(target=publisher, args=([canales]))
    pub_thread.start()
    
    # Ponemos al hilo principal a escuchar 
    for msg in sub.listen():
        if msg['type'] == "message":
            print(f"[SUB] {msg["channel"]}: '{msg["data"]}'")
except KeyboardInterrupt: # 
    print("Subscriber terminado.")


[SUB] ALERTAS:ED-0:SALA-19: 'Nivel de conciencia: Baja'
[SUB] ALERTAS:ED-0:SALA-19: 'Frecuencia respiratoria: Alta'
[SUB] ALERTAS:ED-1:SALA-8: 'Frecuencia respiratoria: Alta'
[SUB] ALERTAS:ED-1:SALA-8: 'Diuresis: Alta'
[SUB] ALERTAS:ED-2:SALA-20: 'Frecuencia respiratoria: Alta'
[SUB] ALERTAS:ED-2:SALA-20: 'Frecuencia respiratoria: Alta'
[SUB] ALERTAS:ED-3:SALA-8: 'Nivel de conciencia: Baja'
[SUB] ALERTAS:ED-3:SALA-8: 'Frecuencia respiratoria: Alta'
[SUB] ALERTAS:ED-2:SALA-11: 'Nivel de conciencia: Alta'
[SUB] ALERTAS:ED-2:SALA-11: 'Frecuencia cardiaca: Alta'
[SUB] ALERTAS:ED-2:SALA-9: 'Frecuencia respiratoria: Alta'
[SUB] ALERTAS:ED-2:SALA-9: 'Frecuencia cardiaca: Baja'
[SUB] ALERTAS:ED-2:SALA-1: 'Diuresis: Baja'
[SUB] ALERTAS:ED-2:SALA-1: 'Nivel de conciencia: Baja'
[SUB] ALERTAS:ED-5:SALA-15: 'Frecuencia cardiaca: Alta'
[SUB] ALERTAS:ED-5:SALA-15: 'Temperatura: Alta'
[PUB] Todas las alertas enviadas!
Subscriber terminado.


### 1.3 Empleados

In [2]:
# Celda 1: Conexión
import redis
import json
from typing import Dict, Any

REDIS_PORT = 6379
# Nos conectamos directamente a la DB=2 para empleados
r_empleados = redis.Redis(host='localhost',
                          port=REDIS_PORT,
                          db=2, # Conectamos a la DB=2
                          decode_responses=True)

# Verificamos la conexión
print(r_empleados.ping())

True


In [3]:
# Celda 2: Comprobar memoria y claves insertadas

# Comprobamos la memoria usada en esta DB
info = r_empleados.info('memory')
print(f"Memoria usada en DB 2: {info['used_memory_human']}")

# Buscamos las claves de nuestros empleados
print("\nClaves de empleados encontradas:")
# SCAN es más seguro que KEYS en producción
for key in r_empleados.scan_iter(match="hospital:madrid:empleado:*", count=10):
    print(key)

Memoria usada en DB 2: 1.30M

Claves de empleados encontradas:
hospital:madrid:empleado:1236
hospital:madrid:empleado:1234
hospital:madrid:empleado:1237
hospital:madrid:empleado:1235


In [4]:
# Celda 3: Obtener el JSON completo de un empleado

# Obtenemos todos los datos del empleado 1234
key_empleado = "hospital:madrid:empleado:1234"

# Usamos .json().get() para traer el objeto completo
# Es el equivalente a JSON.GET hospital:madrid:empleado:1234 $
empleado = r_empleados.json().get(key_empleado)

print(f"Datos del empleado {key_empleado}:")
# Usamos json.dumps para mostrarlo formateado
print(json.dumps(empleado, indent=2, ensure_ascii=False))

Datos del empleado hospital:madrid:empleado:1234:
{
  "nombre": "Dra. Elena Torres",
  "edad": 45,
  "sexo": "Femenino",
  "puesto": "Cirujana Jefa",
  "salario": 85000,
  "departamento": "Cirugía",
  "sala_asignada": "Quirófano 1"
}


In [5]:
# Celda 4: Obtener campos específicos (Usando JSON Path)

key_empleado = "hospital:madrid:empleado:1234"
key_empleado_2 = "hospital:madrid:empleado:1236"

# Para obtener solo el puesto, usamos la ruta '$.puesto'
# El $ es la raíz, el .puesto accede al campo
puesto = r_empleados.json().get(key_empleado, "$.puesto")
print(f"Puesto del empleado 1234: {puesto[0]}") # Accedemos al primer elemento [0]

# Podemos pedir múltiples campos
datos_cardiologo = r_empleados.json().get(key_empleado_2, "$.nombre", "$.departamento", "$.salario")
print(f"Datos del cardiólogo (1236): {datos_cardiologo}")

Puesto del empleado 1234: Cirujana Jefa
Datos del cardiólogo (1236): {'$.departamento': ['Cardiología'], '$.nombre': ['Dr. Miguel López'], '$.salario': [78000]}


In [ ]:
# Celda 5: Actualizar datos de un empleado

key_enfermero = "hospital:madrid:empleado:1235"

# 1. Le damos un aumento al enfermero Carlos Ruiz
# Usamos JSON.NUMINCRBY para sumar 2500 a su salario [cite: 2140]
print(f"Aumentando salario del empleado 1235...")
r_empleados.json().numincrby(key_enfermero, "$.salario", 2500)

# 2. Le cambiamos de sala
# Usamos JSON.SET para actualizar un campo existente [cite: 2133]
print(f"Cambiando de sala al empleado 1235...")
r_empleados.json().set(key_enfermero, "$.sala_asignada", "UCI Planta 5 - Box 3")

# 3. Verificamos los cambios
print("\n--- Datos actualizados del empleado 1235 ---")
datos_actualizados = r_empleados.json().get(key_enfermero, "$.nombre", "$.salario", "$.sala_asignada")
print(json.dumps(datos_actualizados, indent=2, ensure_ascii=False))

Aumentando salario del empleado 1235...
Cambiando de sala al empleado 1235...

--- Datos actualizados del empleado 1235 ---
{
  "$.salario": [
    44500
  ],
  "$.nombre": [
    "Enfermero Carlos Ruiz"
  ],
  "$.sala_asignada": [
    "UCI Planta 5 - Box 3"
  ]
}


## 2. Consultas sobre redis cloud 